In [2]:
import os
import numpy as np
import pandas as pd
import mne
from scipy.stats import ttest_ind, spearmanr

# -------------------------------
# Configuration
# -------------------------------
BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
SUBJECTS = [f"sub-{i:03d}" for i in range(1, 101)]  # adjust as needed

# Electrode mapping per band
ELECTRODES = {
    'delta': ['P3', 'Pz', 'P4', 'P8', 'O1'],
    'theta': ['Cz', 'Pz'],
    'alpha': ['C3', 'P4'],
    'beta':  ['Fp2', 'F8']
}

# Load metadata
meta = pd.read_csv(os.path.join(BASE_DIR, "participants.csv"))
meta = meta[meta['GROUP'] == 'PD'].copy()  # PD only

# -------------------------------
# Compute functional strength per subject/band
# -------------------------------
fs_results = []

for subj in SUBJECTS:
    if subj not in meta['participant_id'].values:
        continue  # skip if not PD or missing

    subj_dir = os.path.join(BASE_DIR, subj)
    subj_num = subj.split('-')[1]
    epo_file = os.path.join(subj_dir, f"epo_{subj_num}_raw.fif")

    if not os.path.exists(epo_file):
        print(f"{subj}: epochs file missing, skipping")
        continue

    # Load epochs
    epochs = mne.read_epochs(epo_file, preload=True, verbose=False)
    ch_names_upper = [ch.upper() for ch in epochs.ch_names]

    for band, elecs in ELECTRODES.items():
        idx = [ch_names_upper.index(ch) for ch in elecs if ch in ch_names_upper]
        if not idx:
            continue

        # Compute mean absolute amplitude as functional strength proxy
        l_freq, h_freq = {'delta': (1,4), 'theta':(4,8), 'alpha':(8,13), 'beta':(13,30)}[band]
        band_data = epochs.copy().filter(l_freq=l_freq, h_freq=h_freq, verbose=False).get_data()[:, idx, :]
        strength = np.mean(np.abs(band_data))

        fs_results.append({
            'subject': subj,
            'band': band,
            'electrodes': ','.join(elecs),
            'functional_strength': strength,
            'MOCA': meta.loc[meta['participant_id']==subj, 'MOCA'].values[0],
            'impaired': meta.loc[meta['participant_id']==subj, 'MOCA'].values[0] < 26
        })

# -------------------------------
# Convert to DataFrame
# -------------------------------
df_fs = pd.DataFrame(fs_results)

# -------------------------------
# Compute stats per band
# -------------------------------
stats_results = []
for band in ELECTRODES.keys():
    df_band = df_fs[df_fs['band'] == band]
    if len(df_band) < 5:
        continue

    imp_vals = df_band.loc[df_band['impaired'], 'functional_strength']
    unimp_vals = df_band.loc[~df_band['impaired'], 'functional_strength']

    t_stat, p_group = ttest_ind(imp_vals, unimp_vals, equal_var=False)
    rho, p_corr = spearmanr(df_band['functional_strength'], df_band['MOCA'])

    stats_results.append({
        'band': band,
        'n_total': len(df_band),
        'n_impaired': imp_vals.shape[0],
        'n_unimpaired': unimp_vals.shape[0],
        't_stat': t_stat,
        'p_group': p_group,
        'rho_MoCA': rho,
        'p_corr': p_corr,
        'impaired_mean': imp_vals.mean(),
        'impaired_std': imp_vals.std(),
        'unimpaired_mean': unimp_vals.mean(),
        'unimpaired_std': unimp_vals.std()
    })

# -------------------------------
# Save results
# -------------------------------
df_stats = pd.DataFrame(stats_results)
df_stats = df_stats.sort_values('p_corr')
output_file = os.path.join(BASE_DIR, "electrode_specific_functional_strength_stats_PD.csv")
df_stats.to_csv(output_file, index=False)
print(df_stats)


C:\Users\User\AppData\Local\Temp\ipykernel_2636\493738992.py:43: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-001\epo_001_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epo_file, preload=True, verbose=False)
C:\Users\User\AppData\Local\Temp\ipykernel_2636\493738992.py:43: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-002\epo_002_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epo_file, preload=True, verbose=False)
C:\Users\User\AppData\Local\Temp\ipykernel_2636\493738992.py:43: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-003\epo_003_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs =

    band  n_total  n_impaired  n_unimpaired    t_stat   p_group  rho_MoCA  \
0  delta      100          53            47  2.370777  0.019716 -0.212064   
1  alpha      100          53            47  0.969290  0.334801 -0.081349   
2   beta      100          53            47 -0.308872  0.758202  0.054677   

     p_corr  impaired_mean  impaired_std  unimpaired_mean  unimpaired_std  
0  0.034164       0.000002  1.161382e-06         0.000002    9.630775e-07  
1  0.421051       0.000003  1.316069e-06         0.000002    1.258809e-06  
2  0.588990       0.000003  8.961444e-07         0.000003    1.256845e-06  


In [1]:
import os
import numpy as np
import pandas as pd
import mne

# -------------------------------
# Configuration
# -------------------------------
BASE_DIR = r"C:\Users\User\Documents\EEG_Project\rEEG"
INPUT_MATRIX = "ML_Feature_Matrix.csv"

# Specifically targeting Delta Significance
DELTA_ELECTRODES = ['P3', 'PZ', 'P4', 'P8', 'O1']
L_FREQ, H_FREQ = 1, 4

# -------------------------------
# Update ML Matrix
# -------------------------------
if not os.path.exists(INPUT_MATRIX):
    print(f"Error: {INPUT_MATRIX} not found.")
else:
    master_df = pd.read_csv(INPUT_MATRIX)
    delta_fs_vals = []

    print("Computing Delta Functional Strength (Posterior ROI)...")

    for subj in master_df['participant_id']:
        subj_num = subj.split('-')[-1]
        epo_file = os.path.join(BASE_DIR, subj, f"epo_{subj_num}_raw.fif")
        
        if os.path.exists(epo_file):
            try:
                # Load and filter for Delta
                epochs = mne.read_epochs(epo_file, preload=True, verbose=False)
                ch_names = [ch.upper() for ch in epochs.ch_names]
                
                # Identify indices for the posterior electrodes
                idx = [ch_names.index(ch) for ch in DELTA_ELECTRODES if ch in ch_names]
                
                if idx:
                    # Filter and extract data for ROI
                    # We compute the mean of the absolute amplitude as 'strength'
                    band_data = epochs.copy().filter(l_freq=L_FREQ, h_freq=H_FREQ, verbose=False).get_data()[:, idx, :]
                    strength = np.mean(np.abs(band_data))
                    delta_fs_vals.append(strength)
                else:
                    delta_fs_vals.append(np.nan)
            except Exception as e:
                print(f"Skipping {subj} due to error: {e}")
                delta_fs_vals.append(np.nan)
        else:
            delta_fs_vals.append(np.nan)

    # Append new feature
    master_df['Feature_Delta_Functional_Strength_Posterior'] = delta_fs_vals

    # Save final matrix
    master_df.to_csv(INPUT_MATRIX, index=False)
    print("Successfully added Feature_Delta_Functional_Strength_Posterior.")
    print(master_df[['participant_id', 'Feature_Delta_Functional_Strength_Posterior']].head())

Computing Delta Functional Strength (Posterior ROI)...


C:\Users\User\AppData\Local\Temp\ipykernel_36460\3079076845.py:34: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-001\epo_001_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epo_file, preload=True, verbose=False)
C:\Users\User\AppData\Local\Temp\ipykernel_36460\3079076845.py:34: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-002\epo_002_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(epo_file, preload=True, verbose=False)
C:\Users\User\AppData\Local\Temp\ipykernel_36460\3079076845.py:34: RuntimeWarning: This filename (C:\Users\User\Documents\EEG_Project\rEEG\sub-003\epo_003_raw.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ep

Successfully added Feature_Delta_Functional_Strength_Posterior.
  participant_id  Feature_Delta_Functional_Strength_Posterior
0        sub-001                                 1.989518e-06
1        sub-002                                 3.240479e-06
2        sub-003                                 3.641637e-06
3        sub-004                                 5.235990e-07
4        sub-005                                 1.857477e-06
